# Assignment 1

## 1.0 Import and install python libraries

In [2]:
# import numpy and pandas libraries
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.impute import SimpleImputer
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.tree import DecisionTreeClassifier 
import summarytools
from summarytools import dfSummary
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
# set random seed to ensure that results are repeatable
np.random.seed(1)

## 3.0 Load data 

In [3]:
# load data
Rscores = pd.read_csv("Restaurant_Scores.csv")

## 4.0 Conduct initial exploration of the data

We have a number of input variables and one target variable. For this analysis, the target variable is price.

First, our initial exploration of the data should answer the following questions:
1. How many rows and columns
2. How much of a problem do we have with na's?
3. What types of data are there?
4. What types of data are stored in columns
    1. identify which variables are numeric and may need to be standardized later
    2. identify which variables are categorical and may need to be transformed using and encoders such as one-hot-encoder.
5. Identify errors in the data - this is a common problem with categorical vars where the category is mispelled or spelled differently in some instances.
 

In [4]:
# look at the data
Rscores.head(3) # note that we don't want to dump all the data to the screen

,business_id,business_name,business_address,business_city,business_state,business_postal_code,business_latitude,business_longitude,business_location,business_phone_number,...,inspection_score,inspection_type,violation_id,violation_description,risk_category,Neighborhoods,SF Find Neighborhoods,Current Police Districts,Current Supervisor Districts,Analysis Neighborhoods
0,1000.0,HEUNG YUEN RESTAURANT,3279 22nd St,San Francisco,CA,94110.0,37.755282,-122.420493,POINT (-122.420493 37.755282),NaN,...,74.0,Routine - Unscheduled,1000_20171002_103103,High risk food holding temperature,High Risk,53.0,53.0,3.0,2.0,20.0
1,1000.0,HEUNG YUEN RESTAURANT,3279 22nd St,San Francisco,CA,94110.0,37.755282,-122.420493,POINT (-122.420493 37.755282),NaN,...,74.0,Routine - Unscheduled,1000_20171002_103132,Improper thawing methods,Moderate Risk,53.0,53.0,3.0,2.0,20.0
2,1000.0,HEUNG YUEN RESTAURANT,3279 22nd St,San Francisco,CA,94110.0,37.755282,-122.420493,POINT (-122.420493 37.755282),NaN,...,74.0,Routine - Unscheduled,1000_20171002_103142,Unclean nonfood contact surfaces,Low Risk,53.0,53.0,3.0,2.0,20.0


In [5]:
# generate a basic summary of the data
Rscores.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1997 entries, 0 to 1996
Data columns (total 22 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   business_id                   1093 non-null   float64
 1   business_name                 1093 non-null   object 
 2   business_address              1093 non-null   object 
 3   business_city                 1093 non-null   object 
 4   business_state                1093 non-null   object 
 5   business_postal_code          1047 non-null   float64
 6   business_latitude             1083 non-null   float64
 7   business_longitude            1083 non-null   float64
 8   business_location             1083 non-null   object 
 9   business_phone_number         421 non-null    float64
 10  inspection_id                 1093 non-null   object 
 11  inspection_date               1093 non-null   object 
 12  inspection_score              1023 non-null   float64
 13  ins

In [6]:
# generate a statistical summary of the numeric value in the data
Rscores.describe()

,business_id,business_postal_code,business_latitude,business_longitude,business_phone_number,inspection_score,Neighborhoods,SF Find Neighborhoods,Current Police Districts,Current Supervisor Districts,Analysis Neighborhoods
count,1093.000000,1047.000000,1083.000000,1083.000000,4.210000e+02,1023.000000,1083.000000,1083.000000,1083.000000,1083.000000,1083.000000
mean,5460.571821,94112.823305,37.770276,-122.424310,1.415558e+10,84.466276,55.077562,55.077562,4.996307,5.227147,17.619575
std,8952.679027,8.375412,0.019829,0.024124,4.903537e+05,8.828026,32.631640,32.631640,2.638753,3.307713,10.759738
min,101.000000,94102.000000,37.711185,-122.504572,1.415052e+10,46.000000,5.000000,5.000000,1.000000,1.000000,1.000000
25%,1249.000000,94107.000000,37.755292,-122.433817,1.415543e+10,80.000000,31.000000,31.000000,3.000000,2.000000,8.000000
50%,1584.000000,94110.000000,37.771117,-122.419004,1.415564e+10,86.000000,53.000000,53.000000,4.000000,4.000000,20.000000
75%,10433.000000,94117.000000,37.787098,-122.407671,1.415576e+10,90.000000,84.000000,84.000000,7.000000,9.000000,22.000000
max,101430.000000,94134.000000,37.808352,-122.383536,1.415838e+10,98.000000,115.000000,115.000000,10.000000,11.000000,41.000000


In [7]:
# Check the missing values by summing the total na's for each variable
Rscores.isna().sum()

business_id                      904
business_name                    904
business_address                 904
business_city                    904
business_state                   904
business_postal_code             950
business_latitude                914
business_longitude               914
business_location                914
business_phone_number           1576
inspection_id                    904
inspection_date                  904
inspection_score                 974
inspection_type                  904
violation_id                     904
violation_description            904
risk_category                    904
Neighborhoods                    914
SF Find Neighborhoods            914
Current Police Districts         914
Current Supervisor Districts     914
Analysis Neighborhoods           914
dtype: int64

In [8]:
# create a list of these catagorical variables
category_var_list = list(Rscores.select_dtypes(include='object').columns)
category_var_list

['business_name',
 'business_address',
 'business_city',
 'business_state',
 'business_location',
 'inspection_id',
 'inspection_date',
 'inspection_type',
 'violation_id',
 'violation_description',
 'risk_category']

In [9]:
# explore the categorical variable values - often there are typos here that need to be fixed.
for cat in category_var_list: # generally, we want to avoid for loops and use a functional style (i.e. list comprehension)
    print(f"Category: {cat} Values: {Rscores[cat].unique()}")

Category: business_name Values: ['HEUNG YUEN RESTAURANT' 'Lamas Peruvian Food Truck' 'BIG MOUTH BURGERS'
 'NAYA CAFE' 'THE MEAT HOUSE' 'RINCON PERUANO RESTAURANT'
 'Marshall Elementary School' "CHAVITA'S MEXICAN RESTAURANT"
 'TAKOYAKI YAMACHAN' 'VALENCIA PIZZA & PASTA' 'Morning Brew Cafe'
 'Cable Car Corner' 'NAMU STONEPOT' "AMICI'S EAST COAST PIZZERIA"
 "Cybelle's Front Room" 'IZAKAYA MAYUMI' 'Piaza Pellegrini'
 'PETES BARBEQUE' "Akiko's Sushi Bar" 'LA SANTANECA DE LA MISSION'
 'The Napper Tandy' 'Nijiya Market' 'PHUKET THAI RESTAURANT'
 'TENDERLOIN ELEMENTARY SCHOOL' 'Jewish Community Center' 'IL BORGO'
 'Harvest Urban Market' 'ALAMO SQUARE' 'El Tonayense #1'
 'Good Frikin Chicken'
 'AT&T - FAB FRY BREAD (LOWER CENTER FIELD) [147480]'
 "LOVE 'N HAIGHT COFFEE & DELI" 'CENTRAL COFFEE TEA & SPICE'
 'Cafe La Taza' 'Piraat Pizza' 'PORK STORE CAFE' 'Slanted Door Restaurant'
 'Boxed Foods Company' 'UNIVERSAL CAFE' 'A16' 'SANDWICH PLACE, THE'
 'Rue Lepic' 'ARINELL PIZZA' 'STARBUCKS COFFEE #9

### Summary the findings from our initial evaluation of the data

* We have 12 categorical variables
* We have 15 variables that have missing values


## 5.0 Process the data

* Conduct any data prepartion that should be done *BEFORE* the data split.
* Split the data.
* Conduct any data preparation that should be done *AFTER* the data split.

### 5.1  Conduct any data prepartion that should be done *BEFORE* the data split

Tasks at this stage include:
1. Drop any columns/features 
2. Decide if you with to exclude any observations (rows) due to missing na's.
2. Conduct proper encoding of categorical variables
    1. You can transform them using dummy variable encoding, one-hot-encoding, or label encoding. 

#### Drop any columns/variables we will not be using

In [10]:
# Our target is price; but there are three related price variableds - price, price_gte_150, 
Rscores.drop(['business_id', 'business_city','business_state','business_latitude','business_longitude','inspection_id','business_phone_number'], axis=1, inplace = True)

Since the business city state are same for all we dropping these columns. latitude and longitude are being removed because we already have the address of the restuarant and the data becomes redundant similar way business id since we already have business name. inspection id , business_phone_numbe doesnt effect our output so removing these two too

#### Drop observations with too many NA's

If we want to remove the rows with NA's use the following code that is commented out. For this exercise - we will not drop rows with NA's 

In [11]:
# If we want to remove rows with NA's use the following code:
Rscores.dropna(axis=0, inplace=True)

In [12]:
# verify that there are now no missing values
Rscores.isna().sum()

business_name                   0
business_address                0
business_postal_code            0
business_location               0
inspection_date                 0
inspection_score                0
inspection_type                 0
violation_id                    0
violation_description           0
risk_category                   0
Neighborhoods                   0
SF Find Neighborhoods           0
Current Police Districts        0
Current Supervisor Districts    0
Analysis Neighborhoods          0
dtype: int64

In [13]:
# investigage how many rows remain 
Rscores.shape

(972, 15)

#### Encode our categorical variables

In [14]:
labelencoder = LabelEncoder()
Rscores['business_name'] = labelencoder.fit_transform(Rscores['business_name'])
Rscores['risk_category'] = labelencoder.fit_transform(Rscores['risk_category'])
Rscores['inspection_type'] = labelencoder.fit_transform(Rscores['inspection_type'])
Rscores['business_address'] = labelencoder.fit_transform(Rscores['business_address'])
Rscores['business_location'] = labelencoder.fit_transform(Rscores['business_location'])
Rscores['business_postal_code'] = labelencoder.fit_transform(Rscores['business_postal_code'])
Rscores['violation_description'] = labelencoder.fit_transform(Rscores['violation_description'])
Rscores['inspection_date'] = labelencoder.fit_transform(Rscores['inspection_date'])
Rscores['violation_id'] = labelencoder.fit_transform(Rscores['violation_id'])

Encoding all the categorical variables with label encoder so that the model fitting will be easy.

In [15]:
# explore the dataframe columns to verify encoding and dropped columns
Rscores.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 972 entries, 0 to 1092
Data columns (total 15 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   business_name                 972 non-null    int32  
 1   business_address              972 non-null    int32  
 2   business_postal_code          972 non-null    int64  
 3   business_location             972 non-null    int32  
 4   inspection_date               972 non-null    int32  
 5   inspection_score              972 non-null    float64
 6   inspection_type               972 non-null    int32  
 7   violation_id                  972 non-null    int32  
 8   violation_description         972 non-null    int32  
 9   risk_category                 972 non-null    int32  
 10  Neighborhoods                 972 non-null    float64
 11  SF Find Neighborhoods         972 non-null    float64
 12  Current Police Districts      972 non-null    float64
 13  Curr

In [16]:
Rscores.head(3)

,business_name,business_address,business_postal_code,business_location,inspection_date,inspection_score,inspection_type,violation_id,violation_description,risk_category,Neighborhoods,SF Find Neighborhoods,Current Police Districts,Current Supervisor Districts,Analysis Neighborhoods
0,66,100,7,112,38,74.0,0,0,5,0,53.0,53.0,3.0,2.0,20.0
1,66,100,7,112,38,74.0,0,1,14,2,53.0,53.0,3.0,2.0,20.0
2,66,100,7,112,38,74.0,0,2,42,1,53.0,53.0,3.0,2.0,20.0


### 5.2 Split data (train/test)

In [17]:
# construct datasets for analysis
target = 'risk_category'
predictors = list(Rscores.columns)
predictors.remove(target)
X = Rscores[predictors]
y = Rscores[target]
train_df, test_df = train_test_split(Rscores, test_size=0.3)


In [18]:
# create the training set and the test set 
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.3, random_state=1)


### 5.3  Conduct any data prepartion that should be done *AFTER* the data split

We will look at the following:
1) imput any missing numeric values using the mean of the variable/column
2) remove differences of scale by standardizing the numerica variables

since we dont have null columns we no need to impute the data

# Scaling the data

In [19]:
# create a standard scaler and fit it to the training set of predictors
scaler = preprocessing.StandardScaler()
cols_to_stdize = predictors               
               
# Transform the predictors of training and validation sets
train_df[cols_to_stdize] = scaler.fit_transform(train_df[cols_to_stdize]) # train_predictors is not a numpy array
test_df[cols_to_stdize] = scaler.transform(test_df[cols_to_stdize]) # validation_target is now a series object


# Resampling Data

Since the data is large and contains about 10000 records only selecting the top 500 rows. I have tried fitting using all the data but the model is taking too much time so reduced the sample size to 500

In [20]:
Rscores = Rscores.head(500)

# save data

In [21]:
X_train.to_csv('./data/Rscores_X_train.csv', index=False)
y_train.to_csv('./data/Rscores_y_train.csv', index=False)
X_test.to_csv('./data/Rscores_X_test.csv', index=False)
y_test.to_csv('./data/Rscores_y_test.csv', index=False)